<a href="https://colab.research.google.com/github/riblidezso/DeepLearningCourse/blob/master/nb_01_baby_steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Baby steps in deep learning



### Load useful python packages

In [0]:
# plotting and numerical basics
%pylab inline

In [0]:
# machine learning baselines
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

In [0]:
# keras deep learning framework (with tensorflow backend)
import keras
from keras.models import Model
import keras.layers as kl
import keras.regularizers as kr
import keras.optimizers as ko

# MNIST handwritten digits dataset

In [0]:
from keras.datasets import mnist
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### Check some handwritten digits

In [0]:
i = 4
imshow(x_train[i])
print('Label:', y_train[i] )

### Some more info about the dataset

In [0]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

### Normalize pixel values to 0-1 range

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train/255
x_test = x_test/255

## Define functions for 'simple' machine learning baselines

In [0]:
def train_baseline(model, x_train, y_train, N_train=60000, **kwargs):
  """Train a baseline sklearn model."""
  x_train_flat = x_train.reshape(x_train.shape[0],-1)  # flatten
  clf = model(**kwargs)  # init machine learning model
  clf.fit(x_train_flat[:N_train],y_train[:N_train])  # train it
  return clf


def test_baseline(clf, x_test, y_test):
  """Evaluate a baseline sklearn model."""
  x_test_flat = x_test.reshape(x_test.shape[0],-1)  # flatten
  y_pred = clf.predict(x_test_flat)  # make predictions
  acc = np.equal(y_pred, y_test).mean()  # calculate accuracy
  print(clf.__class__.__name__, 'accuracy',acc)

### Test logistic regression

[Logistic regression](https://en.wikipedia.org/wiki/Logistic_regression)

In [0]:
%%time
clf = train_baseline(LogisticRegression, x_train, y_train)
test_baseline(clf, x_test, y_test)

### Test Random Forest

[Random Forest](https://en.wikipedia.org/wiki/Random_forest)

In [0]:
%%time
clf = train_baseline(RandomForestClassifier, x_train, y_train,
                    n_jobs=-1, n_estimators = 300)
test_baseline(clf, x_test, y_test)

### Test a simple neural network

In [0]:
%%time
clf = train_baseline(MLPClassifier, x_train, y_train)
test_baseline(clf, x_test, y_test)

In [0]:
# see the params
clf

### Implement our own neural network for more control!

In [0]:
N1 = 100
inp = kl.Input(shape=(28*28,),name='input')  # the input data tensor
x = kl.Dense(N1, activation='relu')(inp)  # first dense layer
x = kl.Dense(10, activation='softmax')(x)  # prediction layer
clf = Model(inputs=inp, outputs=x)  # define the model
clf.compile(loss='sparse_categorical_crossentropy',optimizer='adam',
           metrics=['accuracy'])
clf.summary()

In [0]:
# Train it
clf.fit(x_train.reshape(x_train.shape[0],-1), y_train, epochs=10,batch_size=256,
        validation_data=(x_test.reshape(x_test.shape[0],-1), y_test))

## Try a larger network? 

In [0]:
N1,N2 = 1024,512
inp = kl.Input(shape=(28*28,),name='input')
x = kl.Dense(N1, activation='relu')(inp)  # first dense layer
x = kl.Dropout(0.25)(x)
x = kl.Dense(N2, activation='relu')(x)  # first dense layer
x = kl.Dropout(0.25)(x)
x = kl.Dense(10, activation='softmax')(x)  #prediction layers
clf = Model(inputs=inp, outputs=x)
clf.compile(loss='sparse_categorical_crossentropy',optimizer='adam',
           metrics=['accuracy'])
clf.summary()

In [0]:
# Train it
clf.fit(x_train.reshape(x_train.shape[0],-1),y_train, epochs=10, batch_size=256,
        validation_data=(x_test.reshape(x_test.shape[0],-1), y_test))

# Convolutional neural networks!

In [0]:
# need to create 1 ' color channel'
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

In [0]:
N1,N2,N3 = 32,64,128
inp = kl.Input(shape=(28,28,1),name='input')
x = kl.Conv2D(N1, kernel_size=(3, 3), activation='relu')(inp)
x = kl.Conv2D(N1, (3, 3), activation='relu')(x)
x = kl.MaxPooling2D(pool_size=(2, 2))(x)
x = kl.Conv2D(N2, (3, 3), activation='relu')(x)
x = kl.Conv2D(N2, (3, 3), activation='relu')(x)
x = kl.MaxPooling2D(pool_size=(2, 2))(x)
x = kl.Dropout(0.25)(x)
x = kl.Flatten()(x)
x = kl.Dense(N3, activation='relu')(x)  # first dense layer
x = kl.Dropout(0.5)(x)
x = kl.Dense(10, activation='softmax')(x)  #prediction layers
clf = Model(inputs=inp, outputs=x)
clf.compile(loss='sparse_categorical_crossentropy',optimizer='adam',
           metrics=['accuracy'])
clf.summary()

In [0]:
# Train it
clf.fit(x_train,y_train, epochs=10, validation_split=0.1, batch_size=256,
        validation_data = (x_test, y_test))

---


# 97.5 or 99.4 meh? Let's see a more impressive example: CIFAR10

2.5% vs 0.6% not so meh btw

[CIFAR10 website](https://www.cs.toronto.edu/~kriz/cifar.html)

In [0]:
from keras.datasets import cifar10
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
mnist_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog',
                 'horse','ship', 'truck']

## Test yourself at home! 

I can do around 96%

In [0]:
i = 7
imshow(x_train[i])
plt.axis('off')
print('Label:', mnist_classes[ int(y_train[i]) ])

#### Some info

In [0]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

#### Normalize data

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train/255
x_test = x_test/255

### Baselines

Logistic regression takes forever!

In [0]:
%%time
clf = train_baseline(RandomForestClassifier, x_train, y_train.flatten(),
                    n_jobs=-1, n_estimators = 100)
test_baseline(clf, x_test, y_test.flatten())

# OK, so this is much harder. Let's see neural nets!

### First a simple MLP 

In [0]:
N1 = 200
inp = kl.Input(shape=(32*32*3,),name='input')  # the input data tensor
x = kl.Dense(N1, activation='relu')(inp)  # first dense layer
x = kl.Dense(10, activation='softmax')(x)  # prediction layer
clf = Model(inputs=inp, outputs=x)  # define the model
clf.compile(loss='sparse_categorical_crossentropy',optimizer='adam',
           metrics=['accuracy'])
print(clf.summary())

# Train it
clf.fit(x_train.reshape(x_train.shape[0],-1), y_train, epochs=10,batch_size=256,
        validation_data=(x_test.reshape(x_test.shape[0],-1), y_test))

### Maybe a larger one?

In [0]:
N1,N2 = 1024,512
inp = kl.Input(shape=(32*32*3,),name='input')
x = kl.Dense(N1, activation='relu')(inp)  # first dense layer
x = kl.Dropout(0.25)(x)
x = kl.Dense(N2, activation='relu')(x)  # first dense layer
x = kl.Dropout(0.25)(x)
x = kl.Dense(10, activation='softmax')(x)  #prediction layers
clf = Model(inputs=inp, outputs=x)
clf.compile(loss='sparse_categorical_crossentropy',optimizer='adam',
           metrics=['accuracy'])
print(clf.summary())

# Train it
clf.fit(x_train.reshape(x_train.shape[0],-1), y_train, epochs=20,batch_size=256,
        validation_data=(x_test.reshape(x_test.shape[0],-1), y_test))

## Convolutional, the same as before?

In [0]:
N1,N2,N3 = 32,64,128
inp = kl.Input(shape=(32,32,3),name='input')
x = kl.Conv2D(N1, kernel_size=(3, 3), activation='relu')(inp)
x = kl.Conv2D(N1, (3, 3), activation='relu')(x)
x = kl.MaxPooling2D(pool_size=(2, 2))(x)
x = kl.Conv2D(N2, (3, 3), activation='relu')(x)
x = kl.Conv2D(N2, (3, 3), activation='relu')(x)
x = kl.MaxPooling2D(pool_size=(2, 2))(x)
x = kl.Dropout(0.25)(x)
x = kl.Flatten()(x) 
x = kl.Dense(N3, activation='relu')(x)  # first dense layer
x = kl.Dropout(0.5)(x)
x = kl.Dense(10, activation='softmax')(x)  #prediction layers
clf = Model(inputs=inp, outputs=x)
clf.compile(loss='sparse_categorical_crossentropy',optimizer='adam',
           metrics=['accuracy'])
clf.summary()

In [0]:
# Train it
clf.fit(x_train,y_train, epochs=30, validation_split=0.1, batch_size=256,
        validation_data = (x_test, y_test))

## State of the art is around 97.88% [ArXiv link](https://arxiv.org/abs/1709.01507)


---

# Finally let's implementone of the state of the art networks: [ResNext](https://arxiv.org/abs/1611.05431)

![resnext](https://cdn-images-1.medium.com/max/1600/1*mdiQTfovOXKnqzfj727b9Q.png)


Does not achieve the described results, tell me if you could fix it :)

In [0]:
#!/usr/bin/env python
"""
Train resnext of cifar10.
The implementation follows the FAIR github repo:
https://github.com/facebookresearch/ResNeXt
which is slightly different than the arxiv report.
Here I use the non-preactivation blocks.
The specfic settings (lr,batch size ) can be changed 
to follow the original values (with enough GPUs, and time).
Author: Dezso Ribli
"""

CARDINALITY = 2  # 16
LR = 0.05  # 0.025
BATCH_SIZE = 64  # 32
EPOCHS_DROP = [150,225]
N_EPOCHS = 250
AUG = True

import keras
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Activation
from keras.layers import BatchNormalization, GlobalAveragePooling2D
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras import optimizers
import math


def resnext(inp, resxt_block, cardinality=4):
    """Return resnext."""
    # inital conv
    x = Conv2D(64, (3, 3), padding='same', kernel_regularizer=l2(5e-4))(inp)
    x = Activation('relu')(BatchNormalization()(x))
    # residual blocks
    x = resxt_blocks(x, resxt_block, cardinality, 64, 256, 1)
    x = resxt_blocks(x, resxt_block, cardinality, 128, 512, 2)
    x = resxt_blocks(x, resxt_block, cardinality, 256, 1024, 2)
    # classifier
    x = GlobalAveragePooling2D()(x)
    x = Dense(10,activation='softmax')(x)
    model = Model(inputs=inp, outputs=x)
    return model


def resxt_blocks(x, resxt_block, cardinality, n_ch1, n_ch2, init_stride, 
                 n_block=3):
    """Perform same size residual blocks."""
    x_shortcut = Conv2D(n_ch2, (1, 1), strides = init_stride, 
                        padding='same', kernel_regularizer=l2(5e-4))(x)
    x_shortcut = BatchNormalization()(x_shortcut)
    # first block
    x = resxt_block(x, x_shortcut, cardinality, n_ch1, n_ch2, init_stride)
    for i in range(n_block-1):  # the other residual blocks
        x = resxt_block(x, x, cardinality, n_ch1, n_ch2)
    return x  


def resxt_block_b(x, x_shortcut, cardinality, n_ch1, n_ch2, init_stride=1):
    """Perform a residual block."""
    groups=[]
    for i in range(cardinality):
        y = Conv2D(n_ch1, (1, 1), strides=init_stride, 
                   kernel_regularizer=l2(5e-4), padding='same')(x)
        y = Activation('relu')(BatchNormalization()(y))
        y = Conv2D(n_ch1, (3, 3), padding='same', 
                   kernel_regularizer=l2(5e-4),)(y)
        y = Activation('relu')(BatchNormalization()(y))
        groups.append(y)
    x = keras.layers.concatenate(groups)
    x = Conv2D(n_ch2, (1, 1), padding='same', 
               kernel_regularizer=l2(5e-4),)(x)
    x = BatchNormalization()(x)
    x = keras.layers.add([x, x_shortcut])
    x = Activation('relu')(x)
    return x   


def norm(x):
    """Normalize images."""
    x = x.astype('float32')
    x[...,0] = (x[...,0] - x[...,0].mean())/x[...,0].std()
    x[...,1] = (x[...,1] - x[...,1].mean())/x[...,1].std()
    x[...,2] = (x[...,2] - x[...,2].mean())/x[...,2].std()
    return x


def step_decay(epoch, base_lr=LR, drop=0.1, epochs_drops=EPOCHS_DROP):
    """Helper for step learning rate decay."""
    lrate = base_lr
    for epoch_drop in epochs_drops:
        lrate *= math.pow(drop,math.floor(epoch/epoch_drop))
        return lrate



# SGD
sgd = optimizers.SGD(lr=LR, decay=0, momentum=0.9, nesterov=True)

# resnext
res = resnext(Input(shape=(32,32,3)), resxt_block_b, CARDINALITY)
res.compile(loss='sparse_categorical_crossentropy',
            optimizer=sgd, metrics=['accuracy'])
print(res.summary())  # print summary

# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = map(norm, (x_train, x_test))  # normalize

# train on generator with standard data augmentation
gen = ImageDataGenerator(width_shift_range=0.125,
                         height_shift_range=0.125,
                         horizontal_flip=True)
train_generator = gen.flow(x_train, y_train,
                           batch_size=BATCH_SIZE)
# train
res.fit_generator(train_generator, epochs=N_EPOCHS,
                  steps_per_epoch = len(x_train)/BATCH_SIZE,
                  validation_data=(x_test, y_test),
                  callbacks=[LearningRateScheduler(step_decay)])